<a href="https://colab.research.google.com/github/prisar/ai_notebooks/blob/main/nb_100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
# Authentication and service account setup
from google.colab import auth
from google.auth import default
import os

# Authenticate with Google Cloud
auth.authenticate_user()

# Set project ID
os.environ['GOOGLE_CLOUD_PROJECT'] = 'mrc-quant-ml'


In [5]:

# Install required packages
!pip install -q google-genai google-cloud-aiplatform

In [10]:
# Import and initialize
from google.genai import Client
from google.genai.types import Part, VideoMetadata, FileData

def summarize_video_chunk(video_uri: str, start_offset: str, end_offset: str, prompt: str = "Analyze this video and provide a summary."):
    """Summarizes a video chunk using the Gemini API."""
    client = Client(
        vertexai=True,
        project="mrc-quant-ml",
        location="us-central1",  # Changed from "global"
    )

    response = client.models.generate_content(
        model="gemini-2.0-flash-exp",  # Updated model name
        contents=[
            Part(
                video_metadata=VideoMetadata(
                    fps=1,
                    start_offset=start_offset,
                    end_offset=end_offset
                ),
                file_data=FileData(
                    file_uri=video_uri,
                    mime_type="video/mp4",
                ),
            ),
            prompt
        ],
    )
    return response.text

# Batch processing optimization
async def process_video_chunks_parallel(video_uri: str, chunk_duration: int = 150, max_workers: int = 4):
    """Process video chunks in parallel for better throughput."""
    import asyncio
    from concurrent.futures import ThreadPoolExecutor

    # Get video duration first (you'll need to implement this)
    total_duration = await get_video_duration(video_uri)

    chunks = [(i, min(i + chunk_duration, total_duration))
              for i in range(0, total_duration, chunk_duration)]

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        tasks = [
            executor.submit(
                summarize_video_chunk,
                video_uri,
                f"{start}s",
                f"{end}s"
            ) for start, end in chunks
        ]

        summaries = [task.result() for task in tasks]

    return summaries

# Example usage with error handling
video_uri = "gs://mrc-quant-ml-video-analysis/videoplayback.mp4"

try:
    summary = summarize_video_chunk(
        video_uri=video_uri,
        start_offset="1s",
        end_offset="150s",
        prompt="Extract key technical concepts and timestamps. Focus on actionable insights."
    )
    print(summary)
except Exception as e:
    print(f"Error: {e}")
    # Fallback to shorter chunk
    summary = summarize_video_chunk(video_uri, "1s", "60s")
    print(summary)

Here are key moments and technical concepts covered in the video:

1. **Testing Live Streams (0:00-0:51):**
   -  The streamer tests the live stream capability on YouTube and X (formerly Twitter). 
   -  They discuss if the live stream is working correctly on each platform.

2. **Live Stream Introduction and Thumbnail Discussion (1:00-2:00):**
   - The streamer transitions to introducing the topic of the stream, “Diffusion vs. Autoregressive”.
   -  They note that the video’s thumbnail was AI-generated using Open AI’s GPT image generator but it misspelled the word autoregressive.

3. **Topic: Diffusion Beats Autoregressive in Data-Constrained Settings (2:00-2:24):**
    - The streamer reveals that they will be discussing a research paper that compares diffusion and autoregressive models in data-constrained environments, coming from Carnegie Mellon University.
    -  They mention a video on the topic from “Three Blue One Brown.”

4. **Referencing an External Video (2:25-2:31):**
   - Th